# Meter應用

## SPM3

In [1]:
import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import time
from struct import *

In [2]:
mbComPort = 'COM5' #your RS-485 port #'/dev/ttyUSB0' for linux(RPi3)
baudrate = 9600
databit = 8
parity = 'N'
stopbit = 1
mbTimeout = 100 # ms

![img](spm3_modbus_Vin.gif)

In [1]:
# 0x1000
0x1000

4096

In [ ]:
# 讀回 V_avg

mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
master = modbus_rtu.RtuMaster(mb_port)
master.set_timeout(mbTimeout/1000.0)

mbId = 1
addr = 4100 # base 0

try:
    # FC3
    rr = master.execute(mbId, cst.READ_INPUT_REGISTERS, addr, 100)
    print('read value:', rr)

    # convert to float:
    # IEEE754 ==> (Hi word Hi Bite, Hi word Lo Byte, Lo word Hi Byte, Lo word Lo Byte)
    try:
        ac_v = unpack('>f', pack('>HH', rr[3], rr[2]))[0] # Vin_avg
        print('ac_v=', ac_v)
    except Exception as e:
        print(e)

except Exception as e:
    print("modbus test Error: " + str(e))


master._do_close()

In [ ]:
# 一次讀回一整片

addr = 4100
try:
    rr = master.execute(mb_id, cst.ANALOG_INPUTS, addr, 100)
    print('rr=', rr)
    print('----')
    data_list = list()
    for i in range(len(rr)//2):
        value = unpack('>f', pack('>HH', rr[i*2+1], rr[i*2]))[0]
        data_list.append(value)
    print(data_list)
    print('----')

    # ac_v_avg
    data['ac_v'] = unpack('>f', pack('>HH', rr[3], rr[2]))[0]
    # ac_i_avg
    data['ac_a'] = unpack('>f', pack('>HH', rr[19], rr[18]))[0]
    # freq
    data['freq'] = unpack('>f', pack('>HH', rr[21], rr[20]))[0]
    # power (kw_total)
    data['power'] = unpack('>f', pack('>HH', rr[29], rr[28]))[0]
    # pf
    data['pf'] = unpack('>f', pack('>HH', rr[47], rr[46]))[0]
    # kWh
    data['kwh'] = unpack('>f', pack('>HH', rr[49], rr[48]))[0]

    print(data)
    break #success-->exit to next
except Exception as e:
    print('poll error', e)

master._do_close()
